# Running maxent on the full GPT-labeled data, as well as for pollinators

### maxent options here: https://groups.google.com/g/maxent/c/yRBlvZ1_9rQ

In [12]:
import pandas as pd
import numpy as np
import rasterio
from IPython.display import Image
from dateutil import parser
import shutil

In [18]:
class Mapper:
    """
    The object central to `smood` (simple mapping of occurrence data).
    """
    def __init__(
        self,
        df,
        run_name="test",
        lat_range=None,
        lon_range=None,
        worldclim_layers=list(range(1, 20)),
        outputs_dir="maxent_outputs/",
        write_outputs=False,
        maxent_path=None,
        worldclim_dir=None,
        ):
        """
        The object central to `smood` (simple mapping of occurrence data).
        Users supply a species name, latitude range, and longitude range, and
        then they can run automated maxent sdms over this.
        Parameters:
        -----------
        species_name (str):
            The name of the species.
            e.g., "Monarda fistulosa"
        lat_range (list, tuple):
            A list of the latitude values, low and high, used as bounds for the map.
            e.g., (30, 50)
            Values must be from the range [-90,90]
            These values are sorted later on, so the order doesn't matter.
        lon_range (list, tuple):
            A list of the longitude values, low and high, used as bounds for the map.
            e.g., (-100, -50)
            Values must be from the range [-180,180]
            These values are sorted later on, so the order doesn't matter.
        worldclim_layers (list):
            A list of the layers to use from worldclim. By default, this list contains
            integers 1 through 19, corresponding to all 19 worldclime layers.
        """

        self.profile = {}

        self.df = df
        
        self.profile['run_name'] = run_name
        if lat_range:
            _ = np.sort(lat_range)
            self.profile['ymin'] = _[0]
            self.profile['ymax'] = _[1]
        else:
            self.profile['ymin'] = None
            self.profile['ymax'] = None

        if lon_range:
            _ = np.sort(lon_range)
            self.profile['xmin'] = _[0]
            self.profile['xmax'] = _[1]
        else:
            self.profile['xmin'] = None
            self.profile['xmax'] = None

        if not maxent_path:
            # make the maxent path give the path to the .jar in the package directory...
            self.maxent_path = os.path.join(self.upper_package_level,
                                            'bins',
                                            'maxent.jar')
        else:
            self.maxent_path = maxent_path

        if not worldclim_dir:
            self.worldclim_dir = os.path.join(self.upper_package_level,
                                              'worldclim')
        else:
            self.worldclim_dir = worldclim_dir

        self.worldclim_dict = {
            1:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_01.tif"),
            2:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_02.tif"),
            3:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_03.tif"),
            4:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_04.tif"),
            5:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_05.tif"),
            6:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_06.tif"),
            7:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_07.tif"),
            8:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_08.tif"),
            9:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_09.tif"),
            10: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_10.tif"),
            11: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_11.tif"),
            12: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_12.tif"),
            13: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_13.tif"),
            14: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_14.tif"),
            15: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_15.tif"),
            16: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_16.tif"),
            17: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_17.tif"),
            18: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_18.tif"),
            19: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_19.tif"),
        }

        if worldclim_layers:
            self.profile['worldclim_layers'] = worldclim_layers

        # name folder for maxent outputs
        self.outputs_dir = outputs_dir

        # name folder for clipped/converted worldclim
        #self.envfiles_dir = os.path.join(self.outputs_dir,
        #                                 "envfiles")
        self.envfiles_dir = os.path.join(self.outputs_dir,
                                         "envfiles")

        self.key = None
        self.write_outputs = write_outputs

    def _get_gbif_occs(self):
        # get the gbif key for our species
        self.occfile = os.path.join(self.outputs_dir,
                                    self.profile['run_name']+".csv")

        # make lists to fill
        self.lats = list(self.df.latitude)
        self.lons = list(self.df.longitude)

        # prepare array to write to csv
        csvarr = np.vstack([np.repeat(self.profile['run_name'], len(self.lons)),
                            self.lons,
                            ["{}{}".format(a_, b_) for a_, b_ in zip(self.lats, 
                                                                     np.repeat('\n', 
                                                                               len(self.lats)
                                                                               )
                                                                     )
                             ]
                            ]).T
        # write occurrence data to csv
        with open(self.occfile, 'w') as f:
            f.write('Species,Longitude,Latitude\n')
            for line in csvarr:
                f.write(",".join(line))

        # make these easier to work with downstream
        self.lons = np.array(self.lons)
        self.lats = np.array(self.lats)

    def _write_env_rasters(self):
        """
        Looks at raw worldclim data, clips it to specified bounding box, and writes the result as ascii.
        """
        # loop through the worldclim master files
        for idx, filepath in enumerate([self.worldclim_dict[layer_int] for layer_int in self.profile['worldclim_layers']]):
            # open with rasterio
            envdata = rasterio.open(filepath, 'r')

            # define a window using lat and lon
            win1 = envdata.window(self.profile['xmin'],
                                  self.profile['ymin'],
                                  self.profile['xmax'],
                                  self.profile['ymax'])

            # read env data from the window
            windowarr = envdata.read(window=win1)[0]

            # get affine transform (this will be used to get cell size)
            aff = envdata.profile['transform']

            # get number of columns in the window
            ncols = windowarr.shape[1]

            # get number of rows in the window
            nrows = windowarr.shape[0]

            # define the lower left corner x coordinate (in degrees)
            xllcorner = self.profile['xmin']

            # define the lower left corner y coordinate (in degrees)
            yllcorner = self.profile['ymin']

            # save the cell size from the affine transform
            cellsize = aff.a

            # record the value corresponding to nodata
            nodata_value = envdata.profile['nodata']

            # save ascii file -- saving array with space delimiter, and metadata as a header
            np.savetxt(os.path.join(self.envfiles_dir,
                                    str(idx)+'.asc'),
                       windowarr,
                       delimiter=' ',
                       comments='',
                       header="".join(['ncols {}\n'.format(ncols),
                                       'nrows {}\n'.format(nrows),
                                       'xllcorner {}\n'.format(xllcorner),
                                       'yllcorner {}\n'.format(yllcorner),
                                       'cellsize {}\n'.format(cellsize),
                                       'nodata_value {}'.format(nodata_value)]))

    def run(self):
        """
        Runs gbif and maxent on the species name and bounds provided.
        """

        # make these directories
        #os.mkdir(self.outputs_dir)
        #os.mkdir(self.envfiles_dir)

        self._get_gbif_occs()
        self._write_env_rasters()

        # run maxent from command line

        mkseq = Maxent(self.maxent_path)
        mkseq.open_subprocess()
        mkseq.feed_maxent(self.envfiles_dir,
                          self.occfile,
                          self.outputs_dir,
                          )
        mkseq.close_subprocess()

        # save png output from maxent
        self.maxent_image = Image(os.path.join(self.outputs_dir,
                                               "plots",
                                               self.profile['run_name']+".png"))

        # save raster output from maxent
        self.density_mat = np.genfromtxt(os.path.join(self.outputs_dir,
                                                      self.profile['run_name']+".asc"),
                                         delimiter=' ',
                                         skip_header=6)
        # remove the nans (maxent saves these as -9999)
        self.density_mat[self.density_mat == -9999] = np.nan

        # remove the outputs, we have what we need in memory
        if not self.write_outputs:
            rmtree(self.outputs_dir)
            
import os
import subprocess as sps


class Maxent:
    """
    Opens a view to seq-gen in a subprocess so that many gene trees can be
    cycled through without the overhead of opening/closing subprocesses.
    """

    def __init__(self,
                 maxent_path):

        # set binary path for conda env and check for binary
        self.binary = maxent_path
        assert os.path.exists(self.binary), (
            "binary {} not found".format(self.binary))

        # call open_subprocess to set the shell
        self.shell = None

    def open_subprocess(self):
        """
        Open a persistent Popen bash shell
        """
        # open
        self.shell = sps.Popen(
            ["bash"], stdin=sps.PIPE, stdout=sps.PIPE, bufsize=0)

    def close_subprocess(self):
        """
        Cleanup and shutdown the subprocess shell.
        """
        self.shell.stdin.close()
        self.shell.terminate()
        self.shell.wait(timeout=1.0)

    def feed_maxent(self, envfiles_dir, occfile, outputs_dir):
        """
        Feed a command string a read results until empty line.
        TODO: allow kwargs to add additional seq-gen args.
        """
        # command string
        cmd = (
            "java -mx512m -jar {} nowarnings environmentallayers={} samplesfile={} outputdirectory={} redoifexists autorun; echo done\n"
            .format(self.binary, envfiles_dir, occfile, outputs_dir)
        )

        # feed to the shell
        self.shell.stdin.write(cmd.encode())
        self.shell.stdin.flush()

        # catch returned results until done\n
        hold = []
        for line in iter(self.shell.stdout.readline, b"done\n"):
            hold.append(line.decode())

# Flowers

In [141]:
dat = pd.read_csv('../data/fulldata_cleaned_matched_GPT_colors.csv')
red_validated = pd.read_csv('../data/validated_FULL_gpt_labeled_REDS_ONLY.csv')

/var/folders/qh/c7rwx84921n2kznn09zb2pnh0000gn/T/ipykernel_2175/945864032.py:1: DtypeWarning: Columns (14,29) have mixed types. Specify dtype option on import or set low_memory=False.
  dat = pd.read_csv('../data/fulldata_cleaned_matched_GPT_colors.csv')


### Screen out all of the GPT-labeled reds that aren't truly red

In [142]:
for binom in red_validated.binomial[red_validated.validated.eq('no')]:
    dat = dat[dat.binomial != binom]

## Red flowers

In [20]:
# do all of the reds
reds = ['red']
for start_day in range(0,351,1):
    subdf = dat[dat.day_of_year.isin(range(start_day,start_day+15))]
    subdf = subdf[subdf.color.isin(reds)]
    mapper = Mapper(subdf,run_name='red'+'_'+str(start_day),lat_range=[24,54],lon_range=[-130,-59],
       outputs_dir='../data/maxent/jan24outputs/red_flowers/',
       maxent_path='../bins/maxent.jar',
       worldclim_dir='../data/worldclim/',write_outputs=True,
        worldclim_layers=[1,2,3,4,5,6,7,10,11,12,13,14,15,16,17,18,19]
      )
    mapper.run()

## White flowers

In [ ]:
# do all of the reds
colors = ['white']
run_name_prefix = 'white'
for start_day in range(0,351,1):
    subdf = inatdata_plus_color[inatdata_plus_color.day_of_year.isin(range(start_day,start_day+15))]
    subdf = subdf[subdf.color.isin(colors)]
    mapper = Mapper(subdf,run_name=run_name_prefix+'_'+str(start_day),lat_range=[24,54],lon_range=[-130,-59],
       outputs_dir='./data/maxent/outputs',
       maxent_path='./bins/maxent.jar',
       worldclim_dir='./data/worldclim/',write_outputs=True,
        worldclim_layers=[1,2,3,4,5,6,7,10,11,12,13,14,15,16,17,18,19]
      )
    mapper.run()

# Now do a linear version of each model... just for kicks

In [25]:
class Mapper:
    """
    The object central to `smood` (simple mapping of occurrence data).
    """
    def __init__(
        self,
        df,
        run_name="test",
        lat_range=None,
        lon_range=None,
        worldclim_layers=list(range(1, 20)),
        outputs_dir="maxent_outputs/",
        write_outputs=False,
        maxent_path=None,
        worldclim_dir=None,
        ):
        """
        The object central to `smood` (simple mapping of occurrence data).
        Users supply a species name, latitude range, and longitude range, and
        then they can run automated maxent sdms over this.
        Parameters:
        -----------
        species_name (str):
            The name of the species.
            e.g., "Monarda fistulosa"
        lat_range (list, tuple):
            A list of the latitude values, low and high, used as bounds for the map.
            e.g., (30, 50)
            Values must be from the range [-90,90]
            These values are sorted later on, so the order doesn't matter.
        lon_range (list, tuple):
            A list of the longitude values, low and high, used as bounds for the map.
            e.g., (-100, -50)
            Values must be from the range [-180,180]
            These values are sorted later on, so the order doesn't matter.
        worldclim_layers (list):
            A list of the layers to use from worldclim. By default, this list contains
            integers 1 through 19, corresponding to all 19 worldclime layers.
        """

        self.profile = {}

        self.df = df
        
        self.profile['run_name'] = run_name
        if lat_range:
            _ = np.sort(lat_range)
            self.profile['ymin'] = _[0]
            self.profile['ymax'] = _[1]
        else:
            self.profile['ymin'] = None
            self.profile['ymax'] = None

        if lon_range:
            _ = np.sort(lon_range)
            self.profile['xmin'] = _[0]
            self.profile['xmax'] = _[1]
        else:
            self.profile['xmin'] = None
            self.profile['xmax'] = None

        if not maxent_path:
            # make the maxent path give the path to the .jar in the package directory...
            self.maxent_path = os.path.join(self.upper_package_level,
                                            'bins',
                                            'maxent.jar')
        else:
            self.maxent_path = maxent_path

        if not worldclim_dir:
            self.worldclim_dir = os.path.join(self.upper_package_level,
                                              'worldclim')
        else:
            self.worldclim_dir = worldclim_dir

        self.worldclim_dict = {
            1:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_01.tif"),
            2:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_02.tif"),
            3:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_03.tif"),
            4:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_04.tif"),
            5:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_05.tif"),
            6:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_06.tif"),
            7:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_07.tif"),
            8:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_08.tif"),
            9:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_09.tif"),
            10: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_10.tif"),
            11: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_11.tif"),
            12: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_12.tif"),
            13: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_13.tif"),
            14: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_14.tif"),
            15: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_15.tif"),
            16: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_16.tif"),
            17: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_17.tif"),
            18: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_18.tif"),
            19: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_19.tif"),
        }

        if worldclim_layers:
            self.profile['worldclim_layers'] = worldclim_layers

        # name folder for maxent outputs
        self.outputs_dir = outputs_dir

        # name folder for clipped/converted worldclim
        #self.envfiles_dir = os.path.join(self.outputs_dir,
        #                                 "envfiles")
        self.envfiles_dir = os.path.join(self.outputs_dir,
                                         "envfiles")

        self.key = None
        self.write_outputs = write_outputs

    def _get_gbif_occs(self):
        # get the gbif key for our species
        self.occfile = os.path.join(self.outputs_dir,
                                    self.profile['run_name']+".csv")

        # make lists to fill
        self.lats = list(self.df.latitude)
        self.lons = list(self.df.longitude)

        # prepare array to write to csv
        csvarr = np.vstack([np.repeat(self.profile['run_name'], len(self.lons)),
                            self.lons,
                            ["{}{}".format(a_, b_) for a_, b_ in zip(self.lats, 
                                                                     np.repeat('\n', 
                                                                               len(self.lats)
                                                                               )
                                                                     )
                             ]
                            ]).T
        # write occurrence data to csv
        with open(self.occfile, 'w') as f:
            f.write('Species,Longitude,Latitude\n')
            for line in csvarr:
                f.write(",".join(line))

        # make these easier to work with downstream
        self.lons = np.array(self.lons)
        self.lats = np.array(self.lats)

    def _write_env_rasters(self):
        """
        Looks at raw worldclim data, clips it to specified bounding box, and writes the result as ascii.
        """
        # loop through the worldclim master files
        for idx, filepath in enumerate([self.worldclim_dict[layer_int] for layer_int in self.profile['worldclim_layers']]):
            # open with rasterio
            envdata = rasterio.open(filepath, 'r')

            # define a window using lat and lon
            win1 = envdata.window(self.profile['xmin'],
                                  self.profile['ymin'],
                                  self.profile['xmax'],
                                  self.profile['ymax'])

            # read env data from the window
            windowarr = envdata.read(window=win1)[0]

            # get affine transform (this will be used to get cell size)
            aff = envdata.profile['transform']

            # get number of columns in the window
            ncols = windowarr.shape[1]

            # get number of rows in the window
            nrows = windowarr.shape[0]

            # define the lower left corner x coordinate (in degrees)
            xllcorner = self.profile['xmin']

            # define the lower left corner y coordinate (in degrees)
            yllcorner = self.profile['ymin']

            # save the cell size from the affine transform
            cellsize = aff.a

            # record the value corresponding to nodata
            nodata_value = envdata.profile['nodata']

            # save ascii file -- saving array with space delimiter, and metadata as a header
            ascname = filepath.split(os.sep)[-1] # take just the name
            np.savetxt(os.path.join(self.envfiles_dir,
                                    ascname+'.asc'),
                       windowarr,
                       delimiter=' ',
                       comments='',
                       header="".join(['ncols {}\n'.format(ncols),
                                       'nrows {}\n'.format(nrows),
                                       'xllcorner {}\n'.format(xllcorner),
                                       'yllcorner {}\n'.format(yllcorner),
                                       'cellsize {}\n'.format(cellsize),
                                       'nodata_value {}'.format(nodata_value)]))

    def run(self):
        """
        Runs gbif and maxent on the species name and bounds provided.
        """

        # make these directories
        #os.mkdir(self.outputs_dir)
        #os.mkdir(self.envfiles_dir)

        self._get_gbif_occs()
        self._write_env_rasters()

        # run maxent from command line

        mkseq = Maxent(self.maxent_path)
        mkseq.open_subprocess()
        mkseq.feed_maxent(self.envfiles_dir,
                          self.occfile,
                          self.outputs_dir,
                          )
        mkseq.close_subprocess()

        # save png output from maxent
        self.maxent_image = Image(os.path.join(self.outputs_dir,
                                               "plots",
                                               self.profile['run_name']+".png"))

        # save raster output from maxent
        self.density_mat = np.genfromtxt(os.path.join(self.outputs_dir,
                                                      self.profile['run_name']+".asc"),
                                         delimiter=' ',
                                         skip_header=6)
        # remove the nans (maxent saves these as -9999)
        self.density_mat[self.density_mat == -9999] = np.nan

        # remove the outputs, we have what we need in memory
        if not self.write_outputs:
            rmtree(self.outputs_dir)
            
import os
import subprocess as sps


class Maxent:
    """
    Opens a view to seq-gen in a subprocess so that many gene trees can be
    cycled through without the overhead of opening/closing subprocesses.
    """

    def __init__(self,
                 maxent_path):

        # set binary path for conda env and check for binary
        self.binary = maxent_path
        assert os.path.exists(self.binary), (
            "binary {} not found".format(self.binary))

        # call open_subprocess to set the shell
        self.shell = None

    def open_subprocess(self):
        """
        Open a persistent Popen bash shell
        """
        # open
        self.shell = sps.Popen(
            ["bash"], stdin=sps.PIPE, stdout=sps.PIPE, bufsize=0)

    def close_subprocess(self):
        """
        Cleanup and shutdown the subprocess shell.
        """
        self.shell.stdin.close()
        self.shell.terminate()
        self.shell.wait(timeout=1.0)

    def feed_maxent(self, envfiles_dir, occfile, outputs_dir):
        """
        Feed a command string a read results until empty line.
        TODO: allow kwargs to add additional seq-gen args.
        """
        # command string
        cmd = (
            "java -mx512m -jar {} nowarnings environmentallayers={} samplesfile={} outputdirectory={} quadratic=false product=false threshold=false hinge=false redoifexists autorun; echo done\n"
            .format(self.binary, envfiles_dir, occfile, outputs_dir)
        )

        # feed to the shell
        self.shell.stdin.write(cmd.encode())
        self.shell.stdin.flush()

        # catch returned results until done\n
        hold = []
        for line in iter(self.shell.stdout.readline, b"done\n"):
            hold.append(line.decode())

In [22]:
# do all of the reds
reds = ['red']
for start_day in range(0,351,1):
    subdf = dat[dat.day_of_year.isin(range(start_day,start_day+15))]
    subdf = subdf[subdf.color.isin(reds)]
    mapper = Mapper(subdf,run_name='red'+'_'+str(start_day),lat_range=[24,54],lon_range=[-130,-59],
       outputs_dir='../data/maxent/jan24outputs/red_flowers_linear/',
       maxent_path='../bins/maxent.jar',
       worldclim_dir='../data/worldclim/',write_outputs=True,
        worldclim_layers=[1,2,3,4,5,6,7,10,11,12,13,14,15,16,17,18,19]
      )
    mapper.run()

In [ ]:
# do all of the reds
colors = ['white']
run_name_prefix = 'white'
for start_day in range(0,351,1):
    subdf = dat[dat.day_of_year.isin(range(start_day,start_day+15))]
    subdf = subdf[subdf.color.isin(colors)]
    mapper = Mapper(subdf,run_name=run_name_prefix+'_'+str(start_day),lat_range=[24,54],lon_range=[-130,-59],
       outputs_dir='../data/maxent/jan24outputs/white_flowers_linear',
       maxent_path='../bins/maxent.jar',
       worldclim_dir='../data/worldclim/',write_outputs=True,
        worldclim_layers=[1,2,3,4,5,6,7,10,11,12,13,14,15,16,17,18,19]
      )
    mapper.run()

## With jackknifing

In [99]:
class Mapper:
    """
    The object central to `smood` (simple mapping of occurrence data).
    """
    def __init__(
        self,
        df,
        run_name="test",
        lat_range=None,
        lon_range=None,
        worldclim_layers=list(range(1, 20)),
        outputs_dir="maxent_outputs/",
        write_outputs=False,
        maxent_path=None,
        worldclim_dir=None,
        ):
        """
        The object central to `smood` (simple mapping of occurrence data).
        Users supply a species name, latitude range, and longitude range, and
        then they can run automated maxent sdms over this.
        Parameters:
        -----------
        species_name (str):
            The name of the species.
            e.g., "Monarda fistulosa"
        lat_range (list, tuple):
            A list of the latitude values, low and high, used as bounds for the map.
            e.g., (30, 50)
            Values must be from the range [-90,90]
            These values are sorted later on, so the order doesn't matter.
        lon_range (list, tuple):
            A list of the longitude values, low and high, used as bounds for the map.
            e.g., (-100, -50)
            Values must be from the range [-180,180]
            These values are sorted later on, so the order doesn't matter.
        worldclim_layers (list):
            A list of the layers to use from worldclim. By default, this list contains
            integers 1 through 19, corresponding to all 19 worldclime layers.
        """

        self.profile = {}

        self.df = df
        
        self.profile['run_name'] = run_name
        if lat_range:
            _ = np.sort(lat_range)
            self.profile['ymin'] = _[0]
            self.profile['ymax'] = _[1]
        else:
            self.profile['ymin'] = None
            self.profile['ymax'] = None

        if lon_range:
            _ = np.sort(lon_range)
            self.profile['xmin'] = _[0]
            self.profile['xmax'] = _[1]
        else:
            self.profile['xmin'] = None
            self.profile['xmax'] = None

        if not maxent_path:
            # make the maxent path give the path to the .jar in the package directory...
            self.maxent_path = os.path.join(self.upper_package_level,
                                            'bins',
                                            'maxent.jar')
        else:
            self.maxent_path = maxent_path

        if not worldclim_dir:
            self.worldclim_dir = os.path.join(self.upper_package_level,
                                              'worldclim')
        else:
            self.worldclim_dir = worldclim_dir

        self.worldclim_dict = {
            1:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_01.tif"),
            2:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_02.tif"),
            3:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_03.tif"),
            4:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_04.tif"),
            5:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_05.tif"),
            6:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_06.tif"),
            7:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_07.tif"),
            8:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_08.tif"),
            9:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_09.tif"),
            10: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_10.tif"),
            11: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_11.tif"),
            12: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_12.tif"),
            13: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_13.tif"),
            14: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_14.tif"),
            15: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_15.tif"),
            16: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_16.tif"),
            17: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_17.tif"),
            18: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_18.tif"),
            19: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_19.tif"),
            20: os.path.join(self.worldclim_dir, "wc2.1_10m_elev.tif"),
        }

        if worldclim_layers:
            self.profile['worldclim_layers'] = worldclim_layers

        # name folder for maxent outputs
        self.outputs_dir = outputs_dir

        # name folder for clipped/converted worldclim
        #self.envfiles_dir = os.path.join(self.outputs_dir,
        #                                 "envfiles")
        self.envfiles_dir = os.path.join(self.outputs_dir,
                                         "envfiles")

        self.key = None
        self.write_outputs = write_outputs

    def _get_gbif_occs(self):
        # get the gbif key for our species
        self.occfile = os.path.join(self.outputs_dir,
                                    self.profile['run_name']+".csv")

        # make lists to fill
        self.lats = list(self.df.latitude)
        self.lons = list(self.df.longitude)

        # prepare array to write to csv
        csvarr = np.vstack([np.repeat(self.profile['run_name'], len(self.lons)),
                            self.lons,
                            ["{}{}".format(a_, b_) for a_, b_ in zip(self.lats, 
                                                                     np.repeat('\n', 
                                                                               len(self.lats)
                                                                               )
                                                                     )
                             ]
                            ]).T
        # write occurrence data to csv
        with open(self.occfile, 'w') as f:
            f.write('Species,Longitude,Latitude\n')
            for line in csvarr:
                f.write(",".join(line))

        # make these easier to work with downstream
        self.lons = np.array(self.lons)
        self.lats = np.array(self.lats)

    def _write_env_rasters(self):
        """
        Looks at raw worldclim data, clips it to specified bounding box, and writes the result as ascii.
        """
        # loop through the worldclim master files
        for idx, filepath in enumerate([self.worldclim_dict[layer_int] for layer_int in self.profile['worldclim_layers']]):
            # open with rasterio
            envdata = rasterio.open(filepath, 'r')

            # define a window using lat and lon
            win1 = envdata.window(self.profile['xmin'],
                                  self.profile['ymin'],
                                  self.profile['xmax'],
                                  self.profile['ymax'])

            # read env data from the window
            windowarr = envdata.read(window=win1)[0]

            # get affine transform (this will be used to get cell size)
            aff = envdata.profile['transform']

            # get number of columns in the window
            ncols = windowarr.shape[1]

            # get number of rows in the window
            nrows = windowarr.shape[0]

            # define the lower left corner x coordinate (in degrees)
            xllcorner = self.profile['xmin']

            # define the lower left corner y coordinate (in degrees)
            yllcorner = self.profile['ymin']

            # save the cell size from the affine transform
            cellsize = aff.a

            # record the value corresponding to nodata
            nodata_value = envdata.profile['nodata']

            # save ascii file -- saving array with space delimiter, and metadata as a header
            ascname = filepath.split(os.sep)[-1] # take just the name
            
            np.savetxt(os.path.join(self.envfiles_dir,
                                    ascname+'.asc'),
                       windowarr,
                       delimiter=' ',
                       comments='',
                       header="".join(['ncols {}\n'.format(ncols),
                                       'nrows {}\n'.format(nrows),
                                       'xllcorner {}\n'.format(xllcorner),
                                       'yllcorner {}\n'.format(yllcorner),
                                       'cellsize {}\n'.format(cellsize),
                                       'nodata_value {}'.format(nodata_value)]))

    def run(self):
        """
        Runs gbif and maxent on the species name and bounds provided.
        """

        # make these directories
        #os.mkdir(self.outputs_dir)
        #os.mkdir(self.envfiles_dir)

        self._get_gbif_occs()
        self._write_env_rasters()

        # run maxent from command line

        mkseq = Maxent(self.maxent_path)
        mkseq.open_subprocess()
        mkseq.feed_maxent(self.envfiles_dir,
                          self.occfile,
                          self.outputs_dir,
                          )
        mkseq.close_subprocess()

        # save png output from maxent
        self.maxent_image = Image(os.path.join(self.outputs_dir,
                                               "plots",
                                               self.profile['run_name']+".png"))

        # save raster output from maxent
        self.density_mat = np.genfromtxt(os.path.join(self.outputs_dir,
                                                      self.profile['run_name']+".asc"),
                                         delimiter=' ',
                                         skip_header=6)
        # remove the nans (maxent saves these as -9999)
        self.density_mat[self.density_mat == -9999] = np.nan

        # remove the outputs, we have what we need in memory
        if not self.write_outputs:
            rmtree(self.outputs_dir)
            
import os
import subprocess as sps


class Maxent:
    """
    Opens a view to seq-gen in a subprocess so that many gene trees can be
    cycled through without the overhead of opening/closing subprocesses.
    """

    def __init__(self,
                 maxent_path):

        # set binary path for conda env and check for binary
        self.binary = maxent_path
        assert os.path.exists(self.binary), (
            "binary {} not found".format(self.binary))

        # call open_subprocess to set the shell
        self.shell = None

    def open_subprocess(self):
        """
        Open a persistent Popen bash shell
        """
        # open
        self.shell = sps.Popen(
            ["bash"], stdin=sps.PIPE, stdout=sps.PIPE, bufsize=0)

    def close_subprocess(self):
        """
        Cleanup and shutdown the subprocess shell.
        """
        self.shell.stdin.close()
        self.shell.terminate()
        self.shell.wait(timeout=1.0)

    def feed_maxent(self, envfiles_dir, occfile, outputs_dir):
        """
        Feed a command string a read results until empty line.
        TODO: allow kwargs to add additional seq-gen args.
        """
        # command string
        cmd = (
            "java -mx512m -jar {} nowarnings environmentallayers={} samplesfile={} outputdirectory={} quadratic=false product=false threshold=false hinge=false jackknife=false randomtestpoints=20 redoifexists autorun; echo done\n"
            .format(self.binary, envfiles_dir, occfile, outputs_dir)
        )

        # feed to the shell
        self.shell.stdin.write(cmd.encode())
        self.shell.stdin.flush()

        # catch returned results until done\n
        hold = []
        for line in iter(self.shell.stdout.readline, b"done\n"):
            hold.append(line.decode())

In [100]:
# do all of the reds
reds = ['red']
for start_day in range(0,351,1):
    subdf = dat[dat.day_of_year.isin(range(start_day,start_day+15))]
    subdf = subdf[subdf.color.isin(reds)]
    mapper = Mapper(subdf,run_name='red'+'_'+str(start_day),lat_range=[24,54],lon_range=[-130,-59],
       outputs_dir='../data/maxent/jan24outputs/red_flowers_linear/',
       maxent_path='../bins/maxent.jar',
       worldclim_dir='../data/worldclim/',write_outputs=True,
        worldclim_layers=[1, # Annual Mean Temperature
                          #2, # Mean Diurnal Range (Mean of monthly (max temp - min temp))
                          #3, # Isothermality (BIO2/BIO7) (×100)
                          #4, # Temperature Seasonality (standard deviation ×100)
                          5, # Max Temperature of Warmest Month
                          6, # Min Temperature of Coldest Month
                          #7, # Temperature Annual Range (BIO5-BIO6)
                          #8, # Mean Temperature of Wettest Quarter
                          #9, # Mean Temperature of Driest Quarter
                          #10, # Mean Temperature of Warmest Quarter
                          #11, # Mean Temperature of Coldest Quarter
                          12, # Annual Precipitation
                          13, # Precipitation of Wettest Month
                          14, # Precipitation of Driest Month
                          #15, # Precipitation Seasonality (Coefficient of Variation)
                          #16, # Precipitation of Wettest Quarter
                          #17, # Precipitation of Driest Quarter
                          #18, # Precipitation of Warmest Quarter
                          #19, # Precipitation of Coldest Quarter
                          20 # NEW elevation (wc 2.1) (from SRTM)
                         ]
      )
    mapper.run()

In [ ]:
# do all of the whites
colors = ['white']
run_name_prefix = 'white'
for start_day in range(0,351,1):
    subdf = dat[dat.day_of_year.isin(range(start_day,start_day+15))]
    subdf = subdf[subdf.color.isin(colors)]
    mapper = Mapper(subdf,run_name=run_name_prefix+'_'+str(start_day),lat_range=[24,54],lon_range=[-130,-59],
       outputs_dir='../data/maxent/jan24outputs/white_flowers_linear',
       maxent_path='../bins/maxent.jar',
       worldclim_dir='../data/worldclim/',write_outputs=True,
        worldclim_layers=[1, # Annual Mean Temperature
                          #2, # Mean Diurnal Range (Mean of monthly (max temp - min temp))
                          #3, # Isothermality (BIO2/BIO7) (×100)
                          #4, # Temperature Seasonality (standard deviation ×100)
                          5, # Max Temperature of Warmest Month
                          6, # Min Temperature of Coldest Month
                          #7, # Temperature Annual Range (BIO5-BIO6)
                          #8, # Mean Temperature of Wettest Quarter
                          #9, # Mean Temperature of Driest Quarter
                          #10, # Mean Temperature of Warmest Quarter
                          #11, # Mean Temperature of Coldest Quarter
                          12, # Annual Precipitation
                          13, # Precipitation of Wettest Month
                          14, # Precipitation of Driest Month
                          #15, # Precipitation Seasonality (Coefficient of Variation)
                          #16, # Precipitation of Wettest Quarter
                          #17, # Precipitation of Driest Quarter
                          #18, # Precipitation of Warmest Quarter
                          #19, # Precipitation of Coldest Quarter
                          20 # NEW elevation (wc 2.1) (from SRTM)
                         ]
      )
    mapper.run()

# Trochilidae

In [59]:
troch_dat = pd.read_csv('../raw_inaturalist_exports/hummingbirds/observations-395877.csv.zip')

In [60]:
days_list = []
for date in troch_dat.observed_on:
    dt = parser.parse(date)
    day_of_year = dt.timetuple().tm_yday
    days_list.append(day_of_year)
    
troch_dat['day_of_year'] = days_list

In [84]:
class Mapper:
    """
    The object central to `smood` (simple mapping of occurrence data).
    """
    def __init__(
        self,
        df,
        run_name="test",
        lat_range=None,
        lon_range=None,
        worldclim_layers=list(range(1, 20)),
        outputs_dir="maxent_outputs/",
        write_outputs=False,
        maxent_path=None,
        worldclim_dir=None,
        ):
        """
        The object central to `smood` (simple mapping of occurrence data).
        Users supply a species name, latitude range, and longitude range, and
        then they can run automated maxent sdms over this.
        Parameters:
        -----------
        species_name (str):
            The name of the species.
            e.g., "Monarda fistulosa"
        lat_range (list, tuple):
            A list of the latitude values, low and high, used as bounds for the map.
            e.g., (30, 50)
            Values must be from the range [-90,90]
            These values are sorted later on, so the order doesn't matter.
        lon_range (list, tuple):
            A list of the longitude values, low and high, used as bounds for the map.
            e.g., (-100, -50)
            Values must be from the range [-180,180]
            These values are sorted later on, so the order doesn't matter.
        worldclim_layers (list):
            A list of the layers to use from worldclim. By default, this list contains
            integers 1 through 19, corresponding to all 19 worldclime layers.
        """

        self.profile = {}

        self.df = df
        
        self.profile['run_name'] = run_name
        if lat_range:
            _ = np.sort(lat_range)
            self.profile['ymin'] = _[0]
            self.profile['ymax'] = _[1]
        else:
            self.profile['ymin'] = None
            self.profile['ymax'] = None

        if lon_range:
            _ = np.sort(lon_range)
            self.profile['xmin'] = _[0]
            self.profile['xmax'] = _[1]
        else:
            self.profile['xmin'] = None
            self.profile['xmax'] = None

        if not maxent_path:
            # make the maxent path give the path to the .jar in the package directory...
            self.maxent_path = os.path.join(self.upper_package_level,
                                            'bins',
                                            'maxent.jar')
        else:
            self.maxent_path = maxent_path

        if not worldclim_dir:
            self.worldclim_dir = os.path.join(self.upper_package_level,
                                              'worldclim')
        else:
            self.worldclim_dir = worldclim_dir

        self.worldclim_dict = {
            1:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_01.tif"),
            2:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_02.tif"),
            3:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_03.tif"),
            4:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_04.tif"),
            5:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_05.tif"),
            6:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_06.tif"),
            7:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_07.tif"),
            8:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_08.tif"),
            9:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_09.tif"),
            10: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_10.tif"),
            11: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_11.tif"),
            12: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_12.tif"),
            13: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_13.tif"),
            14: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_14.tif"),
            15: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_15.tif"),
            16: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_16.tif"),
            17: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_17.tif"),
            18: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_18.tif"),
            19: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_19.tif"),
            20: os.path.join(self.worldclim_dir, "wc2.1_10m_elev.tif"),
        }

        if worldclim_layers:
            self.profile['worldclim_layers'] = worldclim_layers

        # name folder for maxent outputs
        self.outputs_dir = outputs_dir

        # name folder for clipped/converted worldclim
        #self.envfiles_dir = os.path.join(self.outputs_dir,
        #                                 "envfiles")
        self.envfiles_dir = os.path.join(self.outputs_dir,
                                         "envfiles")

        self.key = None
        self.write_outputs = write_outputs

    def _get_gbif_occs(self):
        # get the gbif key for our species
        self.occfile = os.path.join(self.outputs_dir,
                                    self.profile['run_name']+".csv")

        # make lists to fill
        self.lats = list(self.df.latitude)
        self.lons = list(self.df.longitude)

        # prepare array to write to csv
        csvarr = np.vstack([np.repeat(self.profile['run_name'], len(self.lons)),
                            self.lons,
                            ["{}{}".format(a_, b_) for a_, b_ in zip(self.lats, 
                                                                     np.repeat('\n', 
                                                                               len(self.lats)
                                                                               )
                                                                     )
                             ]
                            ]).T
        # write occurrence data to csv
        with open(self.occfile, 'w') as f:
            f.write('Species,Longitude,Latitude\n')
            for line in csvarr:
                f.write(",".join(line))

        # make these easier to work with downstream
        self.lons = np.array(self.lons)
        self.lats = np.array(self.lats)

    def _write_env_rasters(self):
        """
        Looks at raw worldclim data, clips it to specified bounding box, and writes the result as ascii.
        """
        # loop through the worldclim master files
        for idx, filepath in enumerate([self.worldclim_dict[layer_int] for layer_int in self.profile['worldclim_layers']]):
            # open with rasterio
            envdata = rasterio.open(filepath, 'r')

            # define a window using lat and lon
            win1 = envdata.window(self.profile['xmin'],
                                  self.profile['ymin'],
                                  self.profile['xmax'],
                                  self.profile['ymax'])

            # read env data from the window
            windowarr = envdata.read(window=win1)[0]

            # get affine transform (this will be used to get cell size)
            aff = envdata.profile['transform']

            # get number of columns in the window
            ncols = windowarr.shape[1]

            # get number of rows in the window
            nrows = windowarr.shape[0]

            # define the lower left corner x coordinate (in degrees)
            xllcorner = self.profile['xmin']

            # define the lower left corner y coordinate (in degrees)
            yllcorner = self.profile['ymin']

            # save the cell size from the affine transform
            cellsize = aff.a

            # record the value corresponding to nodata
            nodata_value = envdata.profile['nodata']

            # save ascii file -- saving array with space delimiter, and metadata as a header
            ascname = filepath.split(os.sep)[-1] # take just the name
            
            np.savetxt(os.path.join(self.envfiles_dir,
                                    ascname+'.asc'),
                       windowarr,
                       delimiter=' ',
                       comments='',
                       header="".join(['ncols {}\n'.format(ncols),
                                       'nrows {}\n'.format(nrows),
                                       'xllcorner {}\n'.format(xllcorner),
                                       'yllcorner {}\n'.format(yllcorner),
                                       'cellsize {}\n'.format(cellsize),
                                       'nodata_value {}'.format(nodata_value)]))

    def run(self):
        """
        Runs gbif and maxent on the species name and bounds provided.
        """

        # make these directories
        #os.mkdir(self.outputs_dir)
        #os.mkdir(self.envfiles_dir)

        self._get_gbif_occs()
        self._write_env_rasters()

        # run maxent from command line

        mkseq = Maxent(self.maxent_path)
        mkseq.open_subprocess()
        mkseq.feed_maxent(self.envfiles_dir,
                          self.occfile,
                          self.outputs_dir,
                          )
        mkseq.close_subprocess()

        # save png output from maxent
        self.maxent_image = Image(os.path.join(self.outputs_dir,
                                               "plots",
                                               self.profile['run_name']+".png"))

        # save raster output from maxent
        self.density_mat = np.genfromtxt(os.path.join(self.outputs_dir,
                                                      self.profile['run_name']+".asc"),
                                         delimiter=' ',
                                         skip_header=6)
        # remove the nans (maxent saves these as -9999)
        self.density_mat[self.density_mat == -9999] = np.nan

        # remove the outputs, we have what we need in memory
        if not self.write_outputs:
            rmtree(self.outputs_dir)
            
import os
import subprocess as sps


class Maxent:
    """
    Opens a view to seq-gen in a subprocess so that many gene trees can be
    cycled through without the overhead of opening/closing subprocesses.
    """

    def __init__(self,
                 maxent_path):

        # set binary path for conda env and check for binary
        self.binary = maxent_path
        assert os.path.exists(self.binary), (
            "binary {} not found".format(self.binary))

        # call open_subprocess to set the shell
        self.shell = None

    def open_subprocess(self):
        """
        Open a persistent Popen bash shell
        """
        # open
        self.shell = sps.Popen(
            ["bash"], stdin=sps.PIPE, stdout=sps.PIPE, bufsize=0)

    def close_subprocess(self):
        """
        Cleanup and shutdown the subprocess shell.
        """
        self.shell.stdin.close()
        self.shell.terminate()
        self.shell.wait(timeout=1.0)

    def feed_maxent(self, envfiles_dir, occfile, outputs_dir):
        """
        Feed a command string a read results until empty line.
        TODO: allow kwargs to add additional seq-gen args.
        """
        # command string
        cmd = (
            "java -mx512m -jar {} nowarnings environmentallayers={} samplesfile={} outputdirectory={} quadratic=false product=false threshold=false hinge=false jackknife=false redoifexists autorun; echo done\n"
            .format(self.binary, envfiles_dir, occfile, outputs_dir)
        )

        # feed to the shell
        self.shell.stdin.write(cmd.encode())
        self.shell.stdin.flush()

        # catch returned results until done\n
        hold = []
        for line in iter(self.shell.stdout.readline, b"done\n"):
            hold.append(line.decode())

In [85]:
# do all of the hummingbirds
for start_day in range(0,351,1):
    subdf = troch_dat[troch_dat.day_of_year.isin(range(start_day,start_day+15))]
    mapper = Mapper(subdf,run_name='hummingbird'+'_'+str(start_day),lat_range=[24,54],lon_range=[-130,-59],
       outputs_dir='../data/maxent/jan24outputs/hummingbirds/',
       maxent_path='../bins/maxent.jar',
       worldclim_dir='../data/worldclim/',write_outputs=True,
        worldclim_layers=[1, # Annual Mean Temperature
                          #2, # Mean Diurnal Range (Mean of monthly (max temp - min temp))
                          #3, # Isothermality (BIO2/BIO7) (×100)
                          #4, # Temperature Seasonality (standard deviation ×100)
                          5, # Max Temperature of Warmest Month
                          6, # Min Temperature of Coldest Month
                          #7, # Temperature Annual Range (BIO5-BIO6)
                          #8, # Mean Temperature of Wettest Quarter
                          #9, # Mean Temperature of Driest Quarter
                          #10, # Mean Temperature of Warmest Quarter
                          #11, # Mean Temperature of Coldest Quarter
                          12, # Annual Precipitation
                          13, # Precipitation of Wettest Month
                          14, # Precipitation of Driest Month
                          #15, # Precipitation Seasonality (Coefficient of Variation)
                          #16, # Precipitation of Wettest Quarter
                          #17, # Precipitation of Driest Quarter
                          #18, # Precipitation of Warmest Quarter
                          #19, # Precipitation of Coldest Quarter
                          20 # NEW elevation (wc 2.1) (from SRTM)
                         ]
      )
    mapper.run()

# Bombus

In [54]:
# folder where the input files are stored
raw_obs_directory = '../raw_inaturalist_exports/bumblebees/'

In [55]:
# input file names (as downloaded from inaturalist)
filenames = ['observations-396610.csv.zip',
 'observations-396616.csv.zip',
 'observations-396678.csv.zip',
            ]

In [56]:
# Specify the full file paths
file_paths = [os.path.join(raw_obs_directory,i) for i in filenames]

In [57]:
# Initialize an empty DataFrame to hold the concatenated data
bombus_dat = pd.DataFrame()

# Loop through file paths and read each file, then concatenate them into all_data
for file_path in file_paths:
    try:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path,low_memory=False)
        
        # Concatenate the DataFrame from the file with the main DataFrame
        bombus_dat = pd.concat([bombus_dat, df], ignore_index=True)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except pd.errors.EmptyDataError:
        print(f"No data in file: {file_path}")
    except pd.errors.ParserError:
        print(f"Error parsing data from file: {file_path}")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

In [62]:
days_list = []
for date in bombus_dat.observed_on:
    dt = parser.parse(date)
    day_of_year = dt.timetuple().tm_yday
    days_list.append(day_of_year)
    
bombus_dat['day_of_year'] = days_list

In [ ]:
# do all bombus
for start_day in range(0,351,1):
    subdf = bombus_dat[bombus_dat.day_of_year.isin(range(start_day,start_day+15))]
    mapper = Mapper(subdf,run_name='bombus'+'_'+str(start_day),lat_range=[24,54],lon_range=[-130,-59],
       outputs_dir='../data/maxent/jan24outputs/bumblebees/',
       maxent_path='../bins/maxent.jar',
       worldclim_dir='../data/worldclim/',write_outputs=True,
        worldclim_layers=[1, # Annual Mean Temperature
                          #2, # Mean Diurnal Range (Mean of monthly (max temp - min temp))
                          #3, # Isothermality (BIO2/BIO7) (×100)
                          #4, # Temperature Seasonality (standard deviation ×100)
                          5, # Max Temperature of Warmest Month
                          6, # Min Temperature of Coldest Month
                          #7, # Temperature Annual Range (BIO5-BIO6)
                          #8, # Mean Temperature of Wettest Quarter
                          #9, # Mean Temperature of Driest Quarter
                          #10, # Mean Temperature of Warmest Quarter
                          #11, # Mean Temperature of Coldest Quarter
                          12, # Annual Precipitation
                          13, # Precipitation of Wettest Month
                          14, # Precipitation of Driest Month
                          #15, # Precipitation Seasonality (Coefficient of Variation)
                          #16, # Precipitation of Wettest Quarter
                          #17, # Precipitation of Driest Quarter
                          #18, # Precipitation of Warmest Quarter
                          #19, # Precipitation of Coldest Quarter
                          20 # NEW elevation (wc 2.1) (from SRTM)
                         ]
      )
    mapper.run()

# Using just the two predictors

### Make sure to turn on jackknifing

In [97]:
class Mapper:
    """
    The object central to `smood` (simple mapping of occurrence data).
    """
    def __init__(
        self,
        df,
        run_name="test",
        lat_range=None,
        lon_range=None,
        worldclim_layers=list(range(1, 20)),
        outputs_dir="maxent_outputs/",
        write_outputs=False,
        maxent_path=None,
        worldclim_dir=None,
        ):
        """
        The object central to `smood` (simple mapping of occurrence data).
        Users supply a species name, latitude range, and longitude range, and
        then they can run automated maxent sdms over this.
        Parameters:
        -----------
        species_name (str):
            The name of the species.
            e.g., "Monarda fistulosa"
        lat_range (list, tuple):
            A list of the latitude values, low and high, used as bounds for the map.
            e.g., (30, 50)
            Values must be from the range [-90,90]
            These values are sorted later on, so the order doesn't matter.
        lon_range (list, tuple):
            A list of the longitude values, low and high, used as bounds for the map.
            e.g., (-100, -50)
            Values must be from the range [-180,180]
            These values are sorted later on, so the order doesn't matter.
        worldclim_layers (list):
            A list of the layers to use from worldclim. By default, this list contains
            integers 1 through 19, corresponding to all 19 worldclime layers.
        """

        self.profile = {}

        self.df = df
        
        self.profile['run_name'] = run_name
        if lat_range:
            _ = np.sort(lat_range)
            self.profile['ymin'] = _[0]
            self.profile['ymax'] = _[1]
        else:
            self.profile['ymin'] = None
            self.profile['ymax'] = None

        if lon_range:
            _ = np.sort(lon_range)
            self.profile['xmin'] = _[0]
            self.profile['xmax'] = _[1]
        else:
            self.profile['xmin'] = None
            self.profile['xmax'] = None

        if not maxent_path:
            # make the maxent path give the path to the .jar in the package directory...
            self.maxent_path = os.path.join(self.upper_package_level,
                                            'bins',
                                            'maxent.jar')
        else:
            self.maxent_path = maxent_path

        if not worldclim_dir:
            self.worldclim_dir = os.path.join(self.upper_package_level,
                                              'worldclim')
        else:
            self.worldclim_dir = worldclim_dir

        self.worldclim_dict = {
            1:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_01.tif"),
            2:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_02.tif"),
            3:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_03.tif"),
            4:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_04.tif"),
            5:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_05.tif"),
            6:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_06.tif"),
            7:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_07.tif"),
            8:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_08.tif"),
            9:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_09.tif"),
            10: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_10.tif"),
            11: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_11.tif"),
            12: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_12.tif"),
            13: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_13.tif"),
            14: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_14.tif"),
            15: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_15.tif"),
            16: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_16.tif"),
            17: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_17.tif"),
            18: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_18.tif"),
            19: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_19.tif"),
            20: os.path.join(self.worldclim_dir, "wc2.1_10m_elev.tif"),
        }

        if worldclim_layers:
            self.profile['worldclim_layers'] = worldclim_layers
        else:
            self.profile['worldclim_layers'] = []

        # name folder for maxent outputs
        self.outputs_dir = outputs_dir

        # name folder for clipped/converted worldclim
        #self.envfiles_dir = os.path.join(self.outputs_dir,
        #                                 "envfiles")
        self.envfiles_dir = os.path.join(self.outputs_dir,
                                         "envfiles")

        self.key = None
        self.write_outputs = write_outputs

    def _get_gbif_occs(self):
        # get the gbif key for our species
        self.occfile = os.path.join(self.outputs_dir,
                                    self.profile['run_name']+".csv")

        # make lists to fill
        self.lats = list(self.df.latitude)
        self.lons = list(self.df.longitude)

        # prepare array to write to csv
        csvarr = np.vstack([np.repeat(self.profile['run_name'], len(self.lons)),
                            self.lons,
                            ["{}{}".format(a_, b_) for a_, b_ in zip(self.lats, 
                                                                     np.repeat('\n', 
                                                                               len(self.lats)
                                                                               )
                                                                     )
                             ]
                            ]).T
        # write occurrence data to csv
        with open(self.occfile, 'w') as f:
            f.write('Species,Longitude,Latitude\n')
            for line in csvarr:
                f.write(",".join(line))

        # make these easier to work with downstream
        self.lons = np.array(self.lons)
        self.lats = np.array(self.lats)

    def _write_env_rasters(self):
        """
        Looks at raw worldclim data, clips it to specified bounding box, and writes the result as ascii.
        """
        # loop through the worldclim master files
        for idx, filepath in enumerate([self.worldclim_dict[layer_int] for layer_int in self.profile['worldclim_layers']]):
            # open with rasterio
            envdata = rasterio.open(filepath, 'r')

            # define a window using lat and lon
            win1 = envdata.window(self.profile['xmin'],
                                  self.profile['ymin'],
                                  self.profile['xmax'],
                                  self.profile['ymax'])

            # read env data from the window
            windowarr = envdata.read(window=win1)[0]

            # get affine transform (this will be used to get cell size)
            aff = envdata.profile['transform']

            # get number of columns in the window
            ncols = windowarr.shape[1]

            # get number of rows in the window
            nrows = windowarr.shape[0]

            # define the lower left corner x coordinate (in degrees)
            xllcorner = self.profile['xmin']

            # define the lower left corner y coordinate (in degrees)
            yllcorner = self.profile['ymin']

            # save the cell size from the affine transform
            cellsize = aff.a

            # record the value corresponding to nodata
            nodata_value = envdata.profile['nodata']

            # save ascii file -- saving array with space delimiter, and metadata as a header
            ascname = filepath.split(os.sep)[-1] # take just the name
            
            np.savetxt(os.path.join(self.envfiles_dir,
                                    ascname+'.asc'),
                       windowarr,
                       delimiter=' ',
                       comments='',
                       header="".join(['ncols {}\n'.format(ncols),
                                       'nrows {}\n'.format(nrows),
                                       'xllcorner {}\n'.format(xllcorner),
                                       'yllcorner {}\n'.format(yllcorner),
                                       'cellsize {}\n'.format(cellsize),
                                       'nodata_value {}'.format(nodata_value)]))

    def run(self):
        """
        Runs gbif and maxent on the species name and bounds provided.
        """

        # make these directories
        #os.mkdir(self.outputs_dir)
        #os.mkdir(self.envfiles_dir)

        self._get_gbif_occs()
        self._write_env_rasters()

        # run maxent from command line

        mkseq = Maxent(self.maxent_path)
        mkseq.open_subprocess()
        mkseq.feed_maxent(self.envfiles_dir,
                          self.occfile,
                          self.outputs_dir,
                          )
        mkseq.close_subprocess()

        # save png output from maxent
        self.maxent_image = Image(os.path.join(self.outputs_dir,
                                               "plots",
                                               self.profile['run_name']+".png"))

        # save raster output from maxent
        self.density_mat = np.genfromtxt(os.path.join(self.outputs_dir,
                                                      self.profile['run_name']+".asc"),
                                         delimiter=' ',
                                         skip_header=6)
        # remove the nans (maxent saves these as -9999)
        self.density_mat[self.density_mat == -9999] = np.nan

        # remove the outputs, we have what we need in memory
        if not self.write_outputs:
            rmtree(self.outputs_dir)
            
import os
import subprocess as sps


class Maxent:
    """
    Opens a view to seq-gen in a subprocess so that many gene trees can be
    cycled through without the overhead of opening/closing subprocesses.
    """

    def __init__(self,
                 maxent_path):

        # set binary path for conda env and check for binary
        self.binary = maxent_path
        assert os.path.exists(self.binary), (
            "binary {} not found".format(self.binary))

        # call open_subprocess to set the shell
        self.shell = None

    def open_subprocess(self):
        """
        Open a persistent Popen bash shell
        """
        # open
        self.shell = sps.Popen(
            ["bash"], stdin=sps.PIPE, stdout=sps.PIPE, bufsize=0)

    def close_subprocess(self):
        """
        Cleanup and shutdown the subprocess shell.
        """
        self.shell.stdin.close()
        self.shell.terminate()
        self.shell.wait(timeout=1.0)

    def feed_maxent(self, envfiles_dir, occfile, outputs_dir):
        """
        Feed a command string a read results until empty line.
        TODO: allow kwargs to add additional seq-gen args.
        """
        # command string
        cmd = (
            "java -mx512m -jar {} nowarnings environmentallayers={} samplesfile={} outputdirectory={} quadratic=false product=false threshold=false hinge=false jackknife=true randomtestpoints=20 redoifexists autorun; echo done\n"
            .format(self.binary, envfiles_dir, occfile, outputs_dir)
        )

        # feed to the shell
        self.shell.stdin.write(cmd.encode())
        self.shell.stdin.flush()

        # catch returned results until done\n
        hold = []
        for line in iter(self.shell.stdout.readline, b"done\n"):
            hold.append(line.decode())

In [98]:
# do all of the reds
reds = ['red']
bombus_maxent_dir = '../data/maxent/jan24outputs/bumblebees/'
troch_maxent_dir = '../data/maxent/jan24outputs/hummingbirds/'
outputs_dir = '../data/maxent/jan24outputs/red_flowers_troch_bombus/'
envfile_dir = os.path.join(outputs_dir,'envfiles')
for start_day in range(0,351,1):
    subdf = dat[dat.day_of_year.isin(range(start_day,start_day+15))]
    subdf = subdf[subdf.color.isin(reds)]
    
    # copy the proper envfiles to the envfiles folder
    
    # Specify the BOMBUS source file path
    source_file_path = os.path.join(bombus_maxent_dir,'bombus_'+str(start_day)+'.asc')
    # Use shutil.copy() to copy the file
    shutil.copy(source_file_path, envfile_dir)
    
    # Specify the TROCH source file path
    source_file_path = os.path.join(troch_maxent_dir,'hummingbird_'+str(start_day)+'.asc')
    # Use shutil.copy() to copy the file
    shutil.copy(source_file_path, envfile_dir)
    
    mapper = Mapper(subdf,run_name='red'+'_'+str(start_day),lat_range=[24,54],lon_range=[-130,-59],
       outputs_dir=outputs_dir,
       maxent_path='../bins/maxent.jar',
       worldclim_dir='../data/worldclim/',write_outputs=True,
        worldclim_layers=[#1, # Annual Mean Temperature
                          #2, # Mean Diurnal Range (Mean of monthly (max temp - min temp))
                          #3, # Isothermality (BIO2/BIO7) (×100)
                          #4, # Temperature Seasonality (standard deviation ×100)
                          #5, # Max Temperature of Warmest Month
                          #6, # Min Temperature of Coldest Month
                          #7, # Temperature Annual Range (BIO5-BIO6)
                          #8, # Mean Temperature of Wettest Quarter
                          #9, # Mean Temperature of Driest Quarter
                          #10, # Mean Temperature of Warmest Quarter
                          #11, # Mean Temperature of Coldest Quarter
                          #12, # Annual Precipitation
                          #13, # Precipitation of Wettest Month
                          #14, # Precipitation of Driest Month
                          #15, # Precipitation Seasonality (Coefficient of Variation)
                          #16, # Precipitation of Wettest Quarter
                          #17, # Precipitation of Driest Quarter
                          #18, # Precipitation of Warmest Quarter
                          #19, # Precipitation of Coldest Quarter
                          #20 # NEW elevation (wc 2.1) (from SRTM)
                         ]
      )
    mapper.run()
    
    # delete the envfiles for this start day
    for filename in os.listdir(envfile_dir):
        file_path = os.path.join(envfile_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
# do all of the whites
colors = ['white']
run_name_prefix = 'white'

bombus_maxent_dir = '../data/maxent/jan24outputs/bumblebees/'
troch_maxent_dir = '../data/maxent/jan24outputs/hummingbirds/'
outputs_dir = '../data/maxent/jan24outputs/white_flowers_troch_bombus/'
envfile_dir = os.path.join(outputs_dir,'envfiles')


for start_day in range(0,351,1):
    subdf = dat[dat.day_of_year.isin(range(start_day,start_day+15))]
    subdf = subdf[subdf.color.isin(colors)]
    
    # copy the proper envfiles to the envfiles folder
    
    # Specify the BOMBUS source file path
    source_file_path = os.path.join(bombus_maxent_dir,'bombus_'+str(start_day)+'.asc')
    # Use shutil.copy() to copy the file
    shutil.copy(source_file_path, envfile_dir)
    
    # Specify the TROCH source file path
    source_file_path = os.path.join(troch_maxent_dir,'hummingbird_'+str(start_day)+'.asc')
    # Use shutil.copy() to copy the file
    shutil.copy(source_file_path, envfile_dir)
    
    mapper = Mapper(subdf,run_name=run_name_prefix+'_'+str(start_day),lat_range=[24,54],lon_range=[-130,-59],
       outputs_dir=outputs_dir,
       maxent_path='../bins/maxent.jar',
       worldclim_dir='../data/worldclim/',write_outputs=True,
        worldclim_layers=[#1, # Annual Mean Temperature
                          #2, # Mean Diurnal Range (Mean of monthly (max temp - min temp))
                          #3, # Isothermality (BIO2/BIO7) (×100)
                          #4, # Temperature Seasonality (standard deviation ×100)
                          #5, # Max Temperature of Warmest Month
                          #6, # Min Temperature of Coldest Month
                          #7, # Temperature Annual Range (BIO5-BIO6)
                          #8, # Mean Temperature of Wettest Quarter
                          #9, # Mean Temperature of Driest Quarter
                          #10, # Mean Temperature of Warmest Quarter
                          #11, # Mean Temperature of Coldest Quarter
                          #12, # Annual Precipitation
                          #13, # Precipitation of Wettest Month
                          #14, # Precipitation of Driest Month
                          #15, # Precipitation Seasonality (Coefficient of Variation)
                          #16, # Precipitation of Wettest Quarter
                          #17, # Precipitation of Driest Quarter
                          #18, # Precipitation of Warmest Quarter
                          #19, # Precipitation of Coldest Quarter
                          #20 # NEW elevation (wc 2.1) (from SRTM)
                         ]
      )
    mapper.run()
    
    # delete the envfiles for this start day
    for filename in os.listdir(envfile_dir):
        file_path = os.path.join(envfile_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

# With replicates, using bombus and hummingbird maps as predictors

In [105]:
class Mapper:
    """
    The object central to `smood` (simple mapping of occurrence data).
    """
    def __init__(
        self,
        df,
        run_name="test",
        lat_range=None,
        lon_range=None,
        worldclim_layers=list(range(1, 20)),
        outputs_dir="maxent_outputs/",
        write_outputs=False,
        maxent_path=None,
        worldclim_dir=None,
        ):
        """
        The object central to `smood` (simple mapping of occurrence data).
        Users supply a species name, latitude range, and longitude range, and
        then they can run automated maxent sdms over this.
        Parameters:
        -----------
        species_name (str):
            The name of the species.
            e.g., "Monarda fistulosa"
        lat_range (list, tuple):
            A list of the latitude values, low and high, used as bounds for the map.
            e.g., (30, 50)
            Values must be from the range [-90,90]
            These values are sorted later on, so the order doesn't matter.
        lon_range (list, tuple):
            A list of the longitude values, low and high, used as bounds for the map.
            e.g., (-100, -50)
            Values must be from the range [-180,180]
            These values are sorted later on, so the order doesn't matter.
        worldclim_layers (list):
            A list of the layers to use from worldclim. By default, this list contains
            integers 1 through 19, corresponding to all 19 worldclime layers.
        """

        self.profile = {}

        self.df = df
        
        self.profile['run_name'] = run_name
        if lat_range:
            _ = np.sort(lat_range)
            self.profile['ymin'] = _[0]
            self.profile['ymax'] = _[1]
        else:
            self.profile['ymin'] = None
            self.profile['ymax'] = None

        if lon_range:
            _ = np.sort(lon_range)
            self.profile['xmin'] = _[0]
            self.profile['xmax'] = _[1]
        else:
            self.profile['xmin'] = None
            self.profile['xmax'] = None

        if not maxent_path:
            # make the maxent path give the path to the .jar in the package directory...
            self.maxent_path = os.path.join(self.upper_package_level,
                                            'bins',
                                            'maxent.jar')
        else:
            self.maxent_path = maxent_path

        if not worldclim_dir:
            self.worldclim_dir = os.path.join(self.upper_package_level,
                                              'worldclim')
        else:
            self.worldclim_dir = worldclim_dir

        self.worldclim_dict = {
            1:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_01.tif"),
            2:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_02.tif"),
            3:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_03.tif"),
            4:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_04.tif"),
            5:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_05.tif"),
            6:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_06.tif"),
            7:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_07.tif"),
            8:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_08.tif"),
            9:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_09.tif"),
            10: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_10.tif"),
            11: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_11.tif"),
            12: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_12.tif"),
            13: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_13.tif"),
            14: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_14.tif"),
            15: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_15.tif"),
            16: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_16.tif"),
            17: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_17.tif"),
            18: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_18.tif"),
            19: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_19.tif"),
            20: os.path.join(self.worldclim_dir, "wc2.1_10m_elev.tif"),
        }

        if worldclim_layers:
            self.profile['worldclim_layers'] = worldclim_layers
        else:
            self.profile['worldclim_layers'] = []

        # name folder for maxent outputs
        self.outputs_dir = outputs_dir

        # name folder for clipped/converted worldclim
        #self.envfiles_dir = os.path.join(self.outputs_dir,
        #                                 "envfiles")
        self.envfiles_dir = os.path.join(self.outputs_dir,
                                         "envfiles")

        self.key = None
        self.write_outputs = write_outputs

    def _get_gbif_occs(self):
        # get the gbif key for our species
        self.occfile = os.path.join(self.outputs_dir,
                                    self.profile['run_name']+".csv")

        # make lists to fill
        self.lats = list(self.df.latitude)
        self.lons = list(self.df.longitude)

        # prepare array to write to csv
        csvarr = np.vstack([np.repeat(self.profile['run_name'], len(self.lons)),
                            self.lons,
                            ["{}{}".format(a_, b_) for a_, b_ in zip(self.lats, 
                                                                     np.repeat('\n', 
                                                                               len(self.lats)
                                                                               )
                                                                     )
                             ]
                            ]).T
        # write occurrence data to csv
        with open(self.occfile, 'w') as f:
            f.write('Species,Longitude,Latitude\n')
            for line in csvarr:
                f.write(",".join(line))

        # make these easier to work with downstream
        self.lons = np.array(self.lons)
        self.lats = np.array(self.lats)

    def _write_env_rasters(self):
        """
        Looks at raw worldclim data, clips it to specified bounding box, and writes the result as ascii.
        """
        # loop through the worldclim master files
        for idx, filepath in enumerate([self.worldclim_dict[layer_int] for layer_int in self.profile['worldclim_layers']]):
            # open with rasterio
            envdata = rasterio.open(filepath, 'r')

            # define a window using lat and lon
            win1 = envdata.window(self.profile['xmin'],
                                  self.profile['ymin'],
                                  self.profile['xmax'],
                                  self.profile['ymax'])

            # read env data from the window
            windowarr = envdata.read(window=win1)[0]

            # get affine transform (this will be used to get cell size)
            aff = envdata.profile['transform']

            # get number of columns in the window
            ncols = windowarr.shape[1]

            # get number of rows in the window
            nrows = windowarr.shape[0]

            # define the lower left corner x coordinate (in degrees)
            xllcorner = self.profile['xmin']

            # define the lower left corner y coordinate (in degrees)
            yllcorner = self.profile['ymin']

            # save the cell size from the affine transform
            cellsize = aff.a

            # record the value corresponding to nodata
            nodata_value = envdata.profile['nodata']

            # save ascii file -- saving array with space delimiter, and metadata as a header
            ascname = filepath.split(os.sep)[-1] # take just the name
            
            np.savetxt(os.path.join(self.envfiles_dir,
                                    ascname+'.asc'),
                       windowarr,
                       delimiter=' ',
                       comments='',
                       header="".join(['ncols {}\n'.format(ncols),
                                       'nrows {}\n'.format(nrows),
                                       'xllcorner {}\n'.format(xllcorner),
                                       'yllcorner {}\n'.format(yllcorner),
                                       'cellsize {}\n'.format(cellsize),
                                       'nodata_value {}'.format(nodata_value)]))

    def run(self):
        """
        Runs gbif and maxent on the species name and bounds provided.
        """

        # make these directories
        #os.mkdir(self.outputs_dir)
        #os.mkdir(self.envfiles_dir)

        self._get_gbif_occs()
        self._write_env_rasters()

        # run maxent from command line

        mkseq = Maxent(self.maxent_path)
        mkseq.open_subprocess()
        mkseq.feed_maxent(self.envfiles_dir,
                          self.occfile,
                          self.outputs_dir,
                          )
        mkseq.close_subprocess()

        # save png output from maxent
        self.maxent_image = Image(os.path.join(self.outputs_dir,
                                               "plots",
                                               self.profile['run_name']+".png"))

        # save raster output from maxent
        #self.density_mat = np.genfromtxt(os.path.join(self.outputs_dir,
        #                                              self.profile['run_name']+".asc"),
        #                                 delimiter=' ',
        #                                 skip_header=6)
        # remove the nans (maxent saves these as -9999)
        #self.density_mat[self.density_mat == -9999] = np.nan

        # remove the outputs, we have what we need in memory
        if not self.write_outputs:
            rmtree(self.outputs_dir)
            
import os
import subprocess as sps


class Maxent:
    """
    Opens a view to seq-gen in a subprocess so that many gene trees can be
    cycled through without the overhead of opening/closing subprocesses.
    """

    def __init__(self,
                 maxent_path):

        # set binary path for conda env and check for binary
        self.binary = maxent_path
        assert os.path.exists(self.binary), (
            "binary {} not found".format(self.binary))

        # call open_subprocess to set the shell
        self.shell = None

    def open_subprocess(self):
        """
        Open a persistent Popen bash shell
        """
        # open
        self.shell = sps.Popen(
            ["bash"], stdin=sps.PIPE, stdout=sps.PIPE, bufsize=0)

    def close_subprocess(self):
        """
        Cleanup and shutdown the subprocess shell.
        """
        self.shell.stdin.close()
        self.shell.terminate()
        self.shell.wait(timeout=1.0)

    def feed_maxent(self, envfiles_dir, occfile, outputs_dir):
        """
        Feed a command string a read results until empty line.
        TODO: allow kwargs to add additional seq-gen args.
        """
        # command string
        cmd = (
            "java -mx512m -jar {} nowarnings environmentallayers={} samplesfile={} outputdirectory={} quadratic=false product=false threshold=false hinge=false jackknife=true randomtestpoints=20 outputgrids=false replicates=10 replicatetype=crossvalidate redoifexists autorun; echo done\n"
            .format(self.binary, envfiles_dir, occfile, outputs_dir)
        )

        # feed to the shell
        self.shell.stdin.write(cmd.encode())
        self.shell.stdin.flush()

        # catch returned results until done\n
        hold = []
        for line in iter(self.shell.stdout.readline, b"done\n"):
            hold.append(line.decode())

In [ ]:
# do all of the reds
reds = ['red']
bombus_maxent_dir = '../data/maxent/jan24outputs/bumblebees/'
troch_maxent_dir = '../data/maxent/jan24outputs/hummingbirds/'
outputs_dir = '../data/maxent/jan24outputs/red_flowers_troch_bombus_10CV/'
envfile_dir = os.path.join(outputs_dir,'envfiles')
for start_day in range(0,351,1):
    subdf = dat[dat.day_of_year.isin(range(start_day,start_day+15))]
    subdf = subdf[subdf.color.isin(reds)]
    
    # copy the proper envfiles to the envfiles folder
    
    # Specify the BOMBUS source file path
    source_file_path = os.path.join(bombus_maxent_dir,'bombus_'+str(start_day)+'.asc')
    # Use shutil.copy() to copy the file
    shutil.copy(source_file_path, envfile_dir)
    
    # Specify the TROCH source file path
    source_file_path = os.path.join(troch_maxent_dir,'hummingbird_'+str(start_day)+'.asc')
    # Use shutil.copy() to copy the file
    shutil.copy(source_file_path, envfile_dir)
    
    mapper = Mapper(subdf,run_name='red'+'_'+str(start_day),lat_range=[24,54],lon_range=[-130,-59],
       outputs_dir=outputs_dir,
       maxent_path='../bins/maxent.jar',
       worldclim_dir='../data/worldclim/',write_outputs=True,
        worldclim_layers=[#1, # Annual Mean Temperature
                          #2, # Mean Diurnal Range (Mean of monthly (max temp - min temp))
                          #3, # Isothermality (BIO2/BIO7) (×100)
                          #4, # Temperature Seasonality (standard deviation ×100)
                          #5, # Max Temperature of Warmest Month
                          #6, # Min Temperature of Coldest Month
                          #7, # Temperature Annual Range (BIO5-BIO6)
                          #8, # Mean Temperature of Wettest Quarter
                          #9, # Mean Temperature of Driest Quarter
                          #10, # Mean Temperature of Warmest Quarter
                          #11, # Mean Temperature of Coldest Quarter
                          #12, # Annual Precipitation
                          #13, # Precipitation of Wettest Month
                          #14, # Precipitation of Driest Month
                          #15, # Precipitation Seasonality (Coefficient of Variation)
                          #16, # Precipitation of Wettest Quarter
                          #17, # Precipitation of Driest Quarter
                          #18, # Precipitation of Warmest Quarter
                          #19, # Precipitation of Coldest Quarter
                          #20 # NEW elevation (wc 2.1) (from SRTM)
                         ]
      )
    mapper.run()
    
    # delete the envfiles for this start day
    for filename in os.listdir(envfile_dir):
        file_path = os.path.join(envfile_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
# do all of the whites
colors = ['white']
run_name_prefix = 'white'

bombus_maxent_dir = '../data/maxent/jan24outputs/bumblebees/'
troch_maxent_dir = '../data/maxent/jan24outputs/hummingbirds/'
outputs_dir = '../data/maxent/jan24outputs/white_flowers_troch_bombus_10CV/'
envfile_dir = os.path.join(outputs_dir,'envfiles')


for start_day in range(0,351,1):
    subdf = dat[dat.day_of_year.isin(range(start_day,start_day+15))]
    subdf = subdf[subdf.color.isin(colors)]
    
    # copy the proper envfiles to the envfiles folder
    
    # Specify the BOMBUS source file path
    source_file_path = os.path.join(bombus_maxent_dir,'bombus_'+str(start_day)+'.asc')
    # Use shutil.copy() to copy the file
    shutil.copy(source_file_path, envfile_dir)
    
    # Specify the TROCH source file path
    source_file_path = os.path.join(troch_maxent_dir,'hummingbird_'+str(start_day)+'.asc')
    # Use shutil.copy() to copy the file
    shutil.copy(source_file_path, envfile_dir)
    
    mapper = Mapper(subdf,run_name=run_name_prefix+'_'+str(start_day),lat_range=[24,54],lon_range=[-130,-59],
       outputs_dir=outputs_dir,
       maxent_path='../bins/maxent.jar',
       worldclim_dir='../data/worldclim/',write_outputs=True,
        worldclim_layers=[#1, # Annual Mean Temperature
                          #2, # Mean Diurnal Range (Mean of monthly (max temp - min temp))
                          #3, # Isothermality (BIO2/BIO7) (×100)
                          #4, # Temperature Seasonality (standard deviation ×100)
                          #5, # Max Temperature of Warmest Month
                          #6, # Min Temperature of Coldest Month
                          #7, # Temperature Annual Range (BIO5-BIO6)
                          #8, # Mean Temperature of Wettest Quarter
                          #9, # Mean Temperature of Driest Quarter
                          #10, # Mean Temperature of Warmest Quarter
                          #11, # Mean Temperature of Coldest Quarter
                          #12, # Annual Precipitation
                          #13, # Precipitation of Wettest Month
                          #14, # Precipitation of Driest Month
                          #15, # Precipitation Seasonality (Coefficient of Variation)
                          #16, # Precipitation of Wettest Quarter
                          #17, # Precipitation of Driest Quarter
                          #18, # Precipitation of Warmest Quarter
                          #19, # Precipitation of Coldest Quarter
                          #20 # NEW elevation (wc 2.1) (from SRTM)
                         ]
      )
    mapper.run()
    
    # delete the envfiles for this start day
    for filename in os.listdir(envfile_dir):
        file_path = os.path.join(envfile_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

# Without replicates, including environmental layers and pollinators

In [151]:
class Mapper:
    """
    The object central to `smood` (simple mapping of occurrence data).
    """
    def __init__(
        self,
        df,
        run_name="test",
        lat_range=None,
        lon_range=None,
        worldclim_layers=list(range(1, 20)),
        outputs_dir="maxent_outputs/",
        write_outputs=False,
        maxent_path=None,
        worldclim_dir=None,
        ):
        """
        The object central to `smood` (simple mapping of occurrence data).
        Users supply a species name, latitude range, and longitude range, and
        then they can run automated maxent sdms over this.
        Parameters:
        -----------
        species_name (str):
            The name of the species.
            e.g., "Monarda fistulosa"
        lat_range (list, tuple):
            A list of the latitude values, low and high, used as bounds for the map.
            e.g., (30, 50)
            Values must be from the range [-90,90]
            These values are sorted later on, so the order doesn't matter.
        lon_range (list, tuple):
            A list of the longitude values, low and high, used as bounds for the map.
            e.g., (-100, -50)
            Values must be from the range [-180,180]
            These values are sorted later on, so the order doesn't matter.
        worldclim_layers (list):
            A list of the layers to use from worldclim. By default, this list contains
            integers 1 through 19, corresponding to all 19 worldclime layers.
        """

        self.profile = {}

        self.df = df
        
        self.profile['run_name'] = run_name
        if lat_range:
            _ = np.sort(lat_range)
            self.profile['ymin'] = _[0]
            self.profile['ymax'] = _[1]
        else:
            self.profile['ymin'] = None
            self.profile['ymax'] = None

        if lon_range:
            _ = np.sort(lon_range)
            self.profile['xmin'] = _[0]
            self.profile['xmax'] = _[1]
        else:
            self.profile['xmin'] = None
            self.profile['xmax'] = None

        if not maxent_path:
            # make the maxent path give the path to the .jar in the package directory...
            self.maxent_path = os.path.join(self.upper_package_level,
                                            'bins',
                                            'maxent.jar')
        else:
            self.maxent_path = maxent_path

        if not worldclim_dir:
            self.worldclim_dir = os.path.join(self.upper_package_level,
                                              'worldclim')
        else:
            self.worldclim_dir = worldclim_dir

        self.worldclim_dict = {
            1:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_01.tif"),
            2:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_02.tif"),
            3:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_03.tif"),
            4:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_04.tif"),
            5:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_05.tif"),
            6:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_06.tif"),
            7:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_07.tif"),
            8:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_08.tif"),
            9:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_09.tif"),
            10: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_10.tif"),
            11: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_11.tif"),
            12: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_12.tif"),
            13: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_13.tif"),
            14: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_14.tif"),
            15: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_15.tif"),
            16: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_16.tif"),
            17: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_17.tif"),
            18: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_18.tif"),
            19: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_19.tif"),
            20: os.path.join(self.worldclim_dir, "wc2.1_10m_elev.tif"),
        }

        if worldclim_layers:
            self.profile['worldclim_layers'] = worldclim_layers
        else:
            self.profile['worldclim_layers'] = []

        # name folder for maxent outputs
        self.outputs_dir = outputs_dir

        # name folder for clipped/converted worldclim
        #self.envfiles_dir = os.path.join(self.outputs_dir,
        #                                 "envfiles")
        self.envfiles_dir = os.path.join(self.outputs_dir,
                                         "envfiles")

        self.key = None
        self.write_outputs = write_outputs

    def _get_gbif_occs(self):
        # get the gbif key for our species
        self.occfile = os.path.join(self.outputs_dir,
                                    self.profile['run_name']+".csv")

        # make lists to fill
        self.lats = list(self.df.latitude)
        self.lons = list(self.df.longitude)

        # prepare array to write to csv
        csvarr = np.vstack([np.repeat(self.profile['run_name'], len(self.lons)),
                            self.lons,
                            ["{}{}".format(a_, b_) for a_, b_ in zip(self.lats, 
                                                                     np.repeat('\n', 
                                                                               len(self.lats)
                                                                               )
                                                                     )
                             ]
                            ]).T
        # write occurrence data to csv
        with open(self.occfile, 'w') as f:
            f.write('Species,Longitude,Latitude\n')
            for line in csvarr:
                f.write(",".join(line))

        # make these easier to work with downstream
        self.lons = np.array(self.lons)
        self.lats = np.array(self.lats)

    def _write_env_rasters(self):
        """
        Looks at raw worldclim data, clips it to specified bounding box, and writes the result as ascii.
        """
        # loop through the worldclim master files
        for idx, filepath in enumerate([self.worldclim_dict[layer_int] for layer_int in self.profile['worldclim_layers']]):
            # open with rasterio
            envdata = rasterio.open(filepath, 'r')

            # define a window using lat and lon
            win1 = envdata.window(self.profile['xmin'],
                                  self.profile['ymin'],
                                  self.profile['xmax'],
                                  self.profile['ymax'])

            # read env data from the window
            windowarr = envdata.read(window=win1)[0]

            # get affine transform (this will be used to get cell size)
            aff = envdata.profile['transform']

            # get number of columns in the window
            ncols = windowarr.shape[1]

            # get number of rows in the window
            nrows = windowarr.shape[0]

            # define the lower left corner x coordinate (in degrees)
            xllcorner = self.profile['xmin']

            # define the lower left corner y coordinate (in degrees)
            yllcorner = self.profile['ymin']

            # save the cell size from the affine transform
            cellsize = aff.a

            # record the value corresponding to nodata
            nodata_value = envdata.profile['nodata']

            # save ascii file -- saving array with space delimiter, and metadata as a header
            ascname = filepath.split(os.sep)[-1] # take just the name
            
            np.savetxt(os.path.join(self.envfiles_dir,
                                    ascname+'.asc'),
                       windowarr,
                       delimiter=' ',
                       comments='',
                       header="".join(['ncols {}\n'.format(ncols),
                                       'nrows {}\n'.format(nrows),
                                       'xllcorner {}\n'.format(xllcorner),
                                       'yllcorner {}\n'.format(yllcorner),
                                       'cellsize {}\n'.format(cellsize),
                                       'nodata_value {}'.format(nodata_value)]))

    def run(self):
        """
        Runs gbif and maxent on the species name and bounds provided.
        """

        # make these directories
        #os.mkdir(self.outputs_dir)
        #os.mkdir(self.envfiles_dir)

        self._get_gbif_occs()
        self._write_env_rasters()

        # run maxent from command line

        mkseq = Maxent(self.maxent_path)
        mkseq.open_subprocess()
        mkseq.feed_maxent(self.envfiles_dir,
                          self.occfile,
                          self.outputs_dir,
                          )
        mkseq.close_subprocess()

        # save png output from maxent
        self.maxent_image = Image(os.path.join(self.outputs_dir,
                                               "plots",
                                               self.profile['run_name']+".png"))

        # save raster output from maxent
        #self.density_mat = np.genfromtxt(os.path.join(self.outputs_dir,
        #                                              self.profile['run_name']+".asc"),
        #                                 delimiter=' ',
        #                                 skip_header=6)
        # remove the nans (maxent saves these as -9999)
        #self.density_mat[self.density_mat == -9999] = np.nan

        # remove the outputs, we have what we need in memory
        if not self.write_outputs:
            rmtree(self.outputs_dir)
            
import os
import subprocess as sps


class Maxent:
    """
    Opens a view to seq-gen in a subprocess so that many gene trees can be
    cycled through without the overhead of opening/closing subprocesses.
    """

    def __init__(self,
                 maxent_path):

        # set binary path for conda env and check for binary
        self.binary = maxent_path
        assert os.path.exists(self.binary), (
            "binary {} not found".format(self.binary))

        # call open_subprocess to set the shell
        self.shell = None

    def open_subprocess(self):
        """
        Open a persistent Popen bash shell
        """
        # open
        self.shell = sps.Popen(
            ["bash"], stdin=sps.PIPE, stdout=sps.PIPE, bufsize=0)

    def close_subprocess(self):
        """
        Cleanup and shutdown the subprocess shell.
        """
        self.shell.stdin.close()
        self.shell.terminate()
        self.shell.wait(timeout=1.0)

    def feed_maxent(self, envfiles_dir, occfile, outputs_dir):
        """
        Feed a command string a read results until empty line.
        TODO: allow kwargs to add additional seq-gen args.
        """
        # command string
        cmd = (
            "java -mx512m -jar {} nowarnings environmentallayers={} samplesfile={} outputdirectory={} quadratic=false product=false threshold=false hinge=false jackknife=true randomtestpoints=20 redoifexists autorun; echo done\n"
            .format(self.binary, envfiles_dir, occfile, outputs_dir)
        )

        # feed to the shell
        self.shell.stdin.write(cmd.encode())
        self.shell.stdin.flush()

        # catch returned results until done\n
        hold = []
        for line in iter(self.shell.stdout.readline, b"done\n"):
            hold.append(line.decode())

In [ ]:
# do all of the whites
colors = ['white']
run_name_prefix = 'white'

bombus_maxent_dir = '../data/maxent/jan24outputs/bumblebees/'
troch_maxent_dir = '../data/maxent/jan24outputs/hummingbirds/'
outputs_dir = '../data/maxent/jan24outputs/white_flowers_troch_bombus_environ/'
envfile_dir = os.path.join(outputs_dir,'envfiles')


for start_day in range(0,351,1):
    subdf = dat[dat.day_of_year.isin(range(start_day,start_day+15))]
    subdf = subdf[subdf.color.isin(colors)]
    
    # copy the proper envfiles to the envfiles folder
    
    # Specify the BOMBUS source file path
    source_file_path = os.path.join(bombus_maxent_dir,'bombus_'+str(start_day)+'.asc')
    # Use shutil.copy() to copy the file
    shutil.copy(source_file_path, envfile_dir)
    
    # Specify the TROCH source file path
    source_file_path = os.path.join(troch_maxent_dir,'hummingbird_'+str(start_day)+'.asc')
    # Use shutil.copy() to copy the file
    shutil.copy(source_file_path, envfile_dir)
    
    mapper = Mapper(subdf,run_name=run_name_prefix+'_'+str(start_day),lat_range=[24,54],lon_range=[-130,-59],
       outputs_dir=outputs_dir,
       maxent_path='../bins/maxent.jar',
       worldclim_dir='../data/worldclim/',write_outputs=True,
        worldclim_layers=[1, # Annual Mean Temperature
                          #2, # Mean Diurnal Range (Mean of monthly (max temp - min temp))
                          #3, # Isothermality (BIO2/BIO7) (×100)
                          #4, # Temperature Seasonality (standard deviation ×100)
                          5, # Max Temperature of Warmest Month
                          6, # Min Temperature of Coldest Month
                          #7, # Temperature Annual Range (BIO5-BIO6)
                          #8, # Mean Temperature of Wettest Quarter
                          #9, # Mean Temperature of Driest Quarter
                          #10, # Mean Temperature of Warmest Quarter
                          #11, # Mean Temperature of Coldest Quarter
                          12, # Annual Precipitation
                          13, # Precipitation of Wettest Month
                          14, # Precipitation of Driest Month
                          #15, # Precipitation Seasonality (Coefficient of Variation)
                          #16, # Precipitation of Wettest Quarter
                          #17, # Precipitation of Driest Quarter
                          #18, # Precipitation of Warmest Quarter
                          #19, # Precipitation of Coldest Quarter
                          20 # NEW elevation (wc 2.1) (from SRTM)
                         ]
      )
    mapper.run()
    
    # delete the envfiles for this start day
    for filename in os.listdir(envfile_dir):
        file_path = os.path.join(envfile_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [220]:
# do all of the reds
reds = ['red']
bombus_maxent_dir = '../data/maxent/jan24outputs/bumblebees/'
troch_maxent_dir = '../data/maxent/jan24outputs/hummingbirds/'
outputs_dir = '../data/maxent/jan24outputs/red_flowers_troch_bombus_environ/'
envfile_dir = os.path.join(outputs_dir,'envfiles')
for start_day in range(0,351,1):
    subdf = dat[dat.day_of_year.isin(range(start_day,start_day+15))]
    subdf = subdf[subdf.color.isin(reds)]
    
    # copy the proper envfiles to the envfiles folder
    
    # Specify the BOMBUS source file path
    source_file_path = os.path.join(bombus_maxent_dir,'bombus_'+str(start_day)+'.asc')
    # Use shutil.copy() to copy the file
    shutil.copy(source_file_path, envfile_dir)
    
    # Specify the TROCH source file path
    source_file_path = os.path.join(troch_maxent_dir,'hummingbird_'+str(start_day)+'.asc')
    # Use shutil.copy() to copy the file
    shutil.copy(source_file_path, envfile_dir)
    
    mapper = Mapper(subdf,run_name='red'+'_'+str(start_day),lat_range=[24,54],lon_range=[-130,-59],
       outputs_dir=outputs_dir,
       maxent_path='../bins/maxent.jar',
       worldclim_dir='../data/worldclim/',write_outputs=True,
        worldclim_layers=[1, # Annual Mean Temperature
                          #2, # Mean Diurnal Range (Mean of monthly (max temp - min temp))
                          #3, # Isothermality (BIO2/BIO7) (×100)
                          #4, # Temperature Seasonality (standard deviation ×100)
                          5, # Max Temperature of Warmest Month
                          6, # Min Temperature of Coldest Month
                          #7, # Temperature Annual Range (BIO5-BIO6)
                          #8, # Mean Temperature of Wettest Quarter
                          #9, # Mean Temperature of Driest Quarter
                          #10, # Mean Temperature of Warmest Quarter
                          #11, # Mean Temperature of Coldest Quarter
                          12, # Annual Precipitation
                          13, # Precipitation of Wettest Month
                          14, # Precipitation of Driest Month
                          #15, # Precipitation Seasonality (Coefficient of Variation)
                          #16, # Precipitation of Wettest Quarter
                          #17, # Precipitation of Driest Quarter
                          #18, # Precipitation of Warmest Quarter
                          #19, # Precipitation of Coldest Quarter
                          20 # NEW elevation (wc 2.1) (from SRTM)
                         ]
      )
    mapper.run()
    
    # delete the envfiles for this start day
    for filename in os.listdir(envfile_dir):
        file_path = os.path.join(envfile_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

# Can we get the percent contribution value from each html?

In [1]:
from bs4 import BeautifulSoup

### red

In [15]:
troch_contributions = []
bombus_contributions = []

for daynum in range(351):
    # Assuming html_content contains your HTML data
    with open('../data/maxent/jan24outputs/red_flowers_troch_bombus_10CV/red_{}.html'.format(daynum)) as f:
        html_content = f.read()

    # Parse the HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the table rows for hummingbird_254 and bombus_254
    rows = soup.find_all('tr', align='right')

    # Extract and print the percent contribution for hummingbird_254 and bombus_254
    environ_tot = 0
    for row in rows:
        cells = row.find_all('td')
        if cells[0].text == 'hummingbird_{}'.format(daynum):
            hummingbird_percent = float(cells[1].text)
        elif cells[0].text == 'bombus_{}'.format(daynum):
            bombus_percent = float(cells[1].text)

    troch_contributions.append(hummingbird_percent)
    bombus_contributions.append(bombus_percent)

In [10]:
import toyplot
import toyplot.pdf
import toyplot.png

In [13]:
llocator = toyplot.locator.Explicit([0,1], ["Hummingbirds", "Bumblebees"])
t = toyplot.matrix(np.array([troch_contributions,bombus_contributions]),
               width=2000,
               height=300,
               llocator=llocator,
               #tshow=False,
               step=50,
               colorshow=True,
               tlabel="Percent Contribution of Hummingbirds vs. Bumblebees vs. Environ to RED Flower Distribution by Day of Year");
#toyplot.pdf.render(t[0],'../troch_bombus_RED_environ.pdf')

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="2000.0px" height="300.0px" viewBox="0 0 2000.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tbb075282a4684807b83d36a9b812db93"> 0 50 100 150 200 250 300 350 Hummingbirds 96.900000 97.500000 96.800000 96.400000 96.700000 96.400000 96.000000 96.500000 96.500000 95.900000 95.600000 96.200000 96.500000 96.300000 96.900000 97.000000 96.900000 96.800000 96.400000 97.300000 96.600000 97.300000 97.000000 97.400000 98.000000 98.400000 98.800000 98.800000 99.100000 99.300000 99.500000 99.700000 99.500000 99.800000 99.600000 99.700000 99.700000 99.700000 99.700000 99.800000 99.900000 99.800000 99.900000 99.900000 100.000000 99.700000 99.300000 95.700000 95.500000 90.600000 80.800000 83.200000 80.200000 71.700000 69.600000 74.200000 69.200000 75.700000 69.000000 65.300000 69.500000 66.800000 70.800000 73.000000 69.600000 72.400000 67.300000 65.600000 69.900000 65.800000 65.400000 65.400000 68.800000 65.400000 67.200000 63.100000 61.800000 60.600000 61.700000 61.100000 63.200000 62.300000 62.100000 66.400000 73.100000 70.300000 72.600000 69.800000 73.700000 72.600000 79.900000 77.900000 76.100000 71.800000 72.200000 71.000000 73.200000 75.200000 75.100000 79.100000 83.200000 83.400000 87.500000 88.100000 88.600000 88.100000 90.700000 92.300000 93.100000 93.300000 94.600000 94.400000 95.700000 96.200000 96.200000 95.800000 97.900000 98.700000 99.400000 99.900000 100.000000 100.000000 100.000000 100.000000 100.000000 100.000000 100.000000 100.000000 100.000000 100.000000 100.000000 100.000000 99.800000 98.700000 95.900000 91.700000 94.800000 92.700000 89.500000 87.900000 85.000000 83.000000 74.500000 82.100000 81.200000 80.900000 80.300000 83.900000 86.300000 88.700000 94.300000 95.700000 98.600000 99.600000 100.000000 100.000000 99.900000 99.500000 99.000000 98.200000 97.200000 97.300000 97.700000 97.500000 97.000000 97.500000 97.500000 97.600000 97.800000 97.200000 97.000000 97.400000 97.500000 98.000000 98.000000 98.400000 98.900000 98.600000 99.000000 99.500000 99.600000 99.700000 99.700000 99.500000 99.200000 99.600000 99.700000 99.900000 99.900000 100.000000 99.800000 99.600000 99.200000 99.400000 99.700000 99.200000 99.100000 98.600000 98.600000 97.700000 94.900000 97.400000 96.100000 96.000000 94.200000 94.100000 94.400000 93.500000 92.500000 90.100000 91.300000 91.600000 93.100000 94.200000 94.100000 97.300000 97.000000 98.100000 98.600000 99.100000 98.900000 98.600000 99.200000 99.200000 99.500000 99.700000 99.300000 98.800000 97.700000 97.500000 95.900000 95.500000 95.600000 96.600000 94.800000 95.000000 95.000000 94.200000 95.200000 95.500000 95.400000 96.300000 96.300000 96.300000 97.300000 97.500000 98.100000 98.100000 97.900000 98.000000 98.100000 98.800000 99.300000 99.400000 99.400000 99.400000 99.500000 99.700000 99.400000 98.800000 98.700000 98.400000 97.300000 97.300000 96.600000 96.800000 96.300000 94.400000 94.100000 93.100000 92.400000 91.600000 89.400000 90.900000 88.600000 87.000000 83.800000 82.200000 81.600000 81.600000 79.200000 77.600000 78.000000 76.300000 74.300000 69.900000 65.300000 61.500000 53.900000 52.900000 50.500000 52.900000 53.700000 53.100000 51.900000 54.700000 55.000000 54.100000 54.400000 57.600000 55.900000 58.000000 59.900000 58.000000 60.700000 64.100000 67.500000 69.900000 71.400000 71.900000 68.300000 68.900000 66.200000 75.600000 76.600000 76.100000 79.600000 84.800000 88.100000 87.100000 83.900000 71.200000 61.700000 62.500000 74.600000 65.500000 64.600000 61.100000 45.300000 49.000000 42.400000 41.200000 41.700000 41.100000 45.400000 32.500000 50.800000 86.100000 98.700000 99.

In [212]:
t

(<toyplot.canvas.Canvas at 0x2c2a87eb0>,
 <toyplot.coordinates.Table at 0x2c2a87e20>)

### white

In [215]:
troch_contributions = []
bombus_contributions = []

for daynum in range(351):
    # Assuming html_content contains your HTML data
    with open('../data/maxent/jan24outputs/white_flowers_troch_bombus_10CV/white_{}.html'.format(daynum)) as f:
        html_content = f.read()

    # Parse the HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the table rows for hummingbird_254 and bombus_254
    rows = soup.find_all('tr', align='right')

    # Extract and print the percent contribution for hummingbird_254 and bombus_254
    for row in rows:
        cells = row.find_all('td')
        if cells[0].text == 'hummingbird_{}'.format(daynum):
            hummingbird_percent = cells[1].text
        elif cells[0].text == 'bombus_{}'.format(daynum):
            bombus_percent = cells[1].text

    troch_contributions.append(hummingbird_percent)
    bombus_contributions.append(bombus_percent)


In [217]:
llocator = toyplot.locator.Explicit([0,1], ["Hummingbirds", "Bumblebees"])
t = toyplot.matrix(np.array([troch_contributions,bombus_contributions]),
               width=2000,
               height=300,
               llocator=llocator,
               #tshow=False,
               step=50,
               colorshow=True,
               tlabel="Percent Contribution of Hummingbirds vs. Bumblebees to WHITE Flower Distribution by Day of Year");
toyplot.pdf.render(t[0],'../troch_bombus_WHITE.pdf')

# Now add in environment

In [30]:
troch_contributions = []
bombus_contributions = []
environ_contributions = []

for daynum in range(351):
    # Assuming html_content contains your HTML data
    with open('../data/maxent/jan24outputs/red_flowers_troch_bombus_environ/red_{}.html'.format(daynum)) as f:
        html_content = f.read()

    # Parse the HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the table rows for hummingbird_254 and bombus_254
    rows = soup.find_all('tr', align='right')

    # Extract and print the percent contribution for hummingbird_254 and bombus_254
    environ_tot = 0
    for row in rows:
        cells = row.find_all('td')
        if cells[0].text == 'hummingbird_{}'.format(daynum):
            hummingbird_percent = float(cells[1].text)
        elif cells[0].text == 'bombus_{}'.format(daynum):
            bombus_percent = float(cells[1].text)
        else:
            environ_tot += float(cells[1].text)

    troch_contributions.append(hummingbird_percent)
    bombus_contributions.append(bombus_percent)
    environ_contributions.append(environ_tot)

In [31]:
llocator = toyplot.locator.Explicit([0,1,2], ["Hummingbirds", "Bumblebees","Environment"])
colormap = toyplot.color.brewer.map("Reds", domain_min=0, domain_max=100, reverse=True)
#toyplot.matrix((matrix, colormap), label="A matrix", colorshow=True);
t = toyplot.matrix((np.array([troch_contributions,bombus_contributions,environ_contributions]),colormap),
               width=2000,
               height=300,
               llocator=llocator,
               #tshow=False,
               step=50,
               colorshow=True,
               tlabel="Percent Contribution of Hummingbirds vs. Bumblebees vs. Environ to RED Flower Distribution by Day of Year");
toyplot.pdf.render(t[0],'../troch_bombus_RED_environ.pdf')

In [32]:
troch_contributions = []
bombus_contributions = []
environ_contributions = []

for daynum in range(351):
    # Assuming html_content contains your HTML data
    with open('../data/maxent/jan24outputs/white_flowers_troch_bombus_environ/white_{}.html'.format(daynum)) as f:
        html_content = f.read()

    # Parse the HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the table rows for hummingbird_254 and bombus_254
    rows = soup.find_all('tr', align='right')

    # Extract and print the percent contribution for hummingbird_254 and bombus_254
    environ_tot = 0
    for row in rows:
        cells = row.find_all('td')
        if cells[0].text == 'hummingbird_{}'.format(daynum):
            hummingbird_percent = float(cells[1].text)
        elif cells[0].text == 'bombus_{}'.format(daynum):
            bombus_percent = float(cells[1].text)
        else:
            environ_tot += float(cells[1].text)

    troch_contributions.append(hummingbird_percent)
    bombus_contributions.append(bombus_percent)
    environ_contributions.append(environ_tot)

In [33]:
llocator = toyplot.locator.Explicit([0,1,2], ["Hummingbirds", "Bumblebees","Environment"])
colormap = toyplot.color.brewer.map("Reds", domain_min=0, domain_max=100, reverse=True)
#toyplot.matrix((matrix, colormap), label="A matrix", colorshow=True);
t = toyplot.matrix((np.array([troch_contributions,bombus_contributions,environ_contributions]),colormap),
               width=2000,
               height=300,
               llocator=llocator,
               #tshow=False,
               step=50,
               colorshow=True,
               tlabel="Percent Contribution of Hummingbirds vs. Bumblebees vs. Environ to WHITE Flower Distribution by Day of Year");
toyplot.pdf.render(t[0],'../troch_bombus_WHITE_environ.pdf')